# t-test 및 다중공선성(VIF) 분석
이 노트북은 `ts2000_y_processed_v1.csv` 데이터를 기반으로 다음 두 가지 분석을 수행합니다:
1. **t-test**: `영업이익_당기` 변수에 대해 `is_defaulted` 그룹 간 평균 차이 비교
2. **VIF (Variance Inflation Factor)**: 다중공선성 진단

In [1]:

import pandas as pd
from scipy.stats import ttest_ind
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

# 데이터 로드
df = pd.read_csv("ts2000_y_processed_v1.csv")


In [2]:

# t-test 분석
group_0 = df[df["is_defaulted"] == 0]["영업이익_당기"].dropna()
group_1 = df[df["is_defaulted"] == 1]["영업이익_당기"].dropna()

t_stat, p_value = ttest_ind(group_0, group_1, equal_var=False)
print("t-statistic:", t_stat)
print("p-value:", p_value)


t-statistic: 8.423209051226166
p-value: 9.017375368375119e-17


**해석**:

- t-test 결과에서 p-value < 0.05이면 두 그룹의 평균 차이는 통계적으로 유의미합니다.
- 본 예시에서는 `영업이익_당기` 값이 `부도기업`과 `비부도기업` 간에 차이가 있음을 시사합니다.

In [3]:

# VIF 분석
vif_vars = ["영업이익_당기", "자산총계_당기", "부채총계_당기"]
vif_data = df[vif_vars].dropna()
vif_data_const = sm.add_constant(vif_data)

vif_result = pd.DataFrame({
    "Variable": vif_data_const.columns,
    "VIF": [variance_inflation_factor(vif_data_const.values, i) for i in range(vif_data_const.shape[1])]
})

print(vif_result)


  Variable       VIF
0    const  1.025354
1  영업이익_당기  2.010699
2  자산총계_당기  7.442136
3  부채총계_당기  5.590672


**해석**:

- VIF 값이 5 이상이면 다중공선성을 의심할 수 있습니다.
- `자산총계_당기`와 `부채총계_당기`는 비교적 높은 VIF 값을 보이며, 다중공선성에 유의해야 합니다.